# **Creating Knowledge base with the geospatial csv dataset** (cities_r2.csv)
## Dataset: [cities_r2.csv](https://docs.google.com/spreadsheets/d/1HIKkB1lGY8IKuzlq7cpJ3Q__GD8S4MC0TLxhgYXa2WY/edit?usp=sharing)


In [ ]:
# Creating a knowledge base
import pandas as p
df = p.read_csv("cities_r2.csv")
knowledge_base = dict()
for i in range(len(df)):
  city_name = (df.iloc[i][0]).lower().strip()
  geospatial_data = f"name_of_city:{df.iloc[i][0]}\n	state_code:{df.iloc[i][1]}\n	state_name:{df.iloc[i][2]}\n	dist_code:{df.iloc[i][3]}\n	population_total:{df.iloc[i][4]}\n	population_male:{df.iloc[i][5]}\n	population_female:{df.iloc[i][6]}\n	0-6_population_total:{df.iloc[i][7]}\n	0-6_population_male:{df.iloc[i][8]}\n	0-6_population_female:{df.iloc[i][9]}\n	literates_total:{df.iloc[i][10]}\n	literates_male:{df.iloc[i][11]}\n	literates_female:{df.iloc[i][12]}\n	sex_ratio:{df.iloc[i][13]}\n	child_sex_ratio:{df.iloc[i][14]}\n	effective_literacy_rate_total:{df.iloc[i][15]}\n	effective_literacy_rate_male:{df.iloc[i][16]}\n	effective_literacy_rate_female:{df.iloc[i][17]}\n	location:{df.iloc[i][18]}\n	total_graduates:{df.iloc[i][19]}\n	male_graduates:{df.iloc[i][20]}\n	female_graduates:{df.iloc[i][21]}"
  knowledge_base[city_name] = geospatial_data

print("Knowledge base created.")

def template(user_input,word):
  rets = f"""Query: {user_input}
  1. Answer to the above query with the provided information
  Information:
  {knowledge_base.get(word)}
  Only include the answer for the query.
  $end$
  """
  return rets

import spacy
nlp = spacy.load('en_core_web_sm')
def preproc(user_input):
    word = user_input
    user_input = user_input.lower()
    doc = nlp(user_input)
    for ent in doc.ents:
        if ent.label_ == "GPE":
            word = ent.text
        elif ent.label_ == "ORG":
          word= ent.text
    each = user_input.lower().split(' ')
    for single_word in each:
      inf = knowledge_base.get(single_word)
      if inf:
        word = single_word

    return template(user_input,word)
#testing
preproc("What is the population of Coimbatore")


Knowledge base created.


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


'Query: what is the population of coimbatore\n  1. Answer to the above query with the provided information\n  Information:\n  name_of_city:Coimbatore \n\tstate_code:33\n\tstate_name:TAMIL NADU\n\tdist_code:31\n\tpopulation_total:1061447\n\tpopulation_male:531017\n\tpopulation_female:530430\n\t0-6_population_total:96261\n\t0-6_population_male:49040\n\t0-6_population_female:47221\n\tliterates_total:885471\n\tliterates_male:457982\n\tliterates_female:427489\n\tsex_ratio:999\n\tchild_sex_ratio:963\n\teffective_literacy_rate_total:91.74\n\teffective_literacy_rate_male:95.02\n\teffective_literacy_rate_female:88.47\n\tlocation:11.0168445,76.9558321\n\ttotal_graduates:168145\n\tmale_graduates:87123\n\tfemale_graduates:81022\n  Only include the answer for the query.\n  $end$\n  '

In [ ]:
"""Installing necessary modules"""
!pip install bitsandbytes torch transformers datasets huggingface_hub peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
notebook_login()


'should remove this'

# Load the base model and tokenizer

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

model-00001-of-00015.safetensors:  56%|#####6    | 944M/1.68G [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model-00004-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00005-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model



In [ ]:
prompt = """
<human>: Tell me about Chennai.
<assistant>:
""".strip()

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<human>: Tell me about Chennai.
<assistant>: Chennai is a city in the south of India, located on the Bay of Bengal. It is the capital of the state of Tamil Nadu and is home to over 4 million people. The city is known for its rich history, culture, and architecture, and is a major center for education, business, and technology.
User 


In [ ]:
prompt = """
<human>: What is the literacy rate of Chennai.
<assistant>:
""".strip()

generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
<human>: What is the literacy rate of Chennai.
<assistant>: Chennai is the capital city of Tamil Nadu and is home to 4 million people. The chennai has literacy of 130 and is known for its bustling streets, rich history, culture, and architecture, and it the hub for technology and education.
User 


# Prep dataset

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)
from datasets import load_dataset
data = load_dataset("csv", data_files="spatialdata.csv")
print(data["train"])
def generate_prompt(data_point):

  ret = f"""
<human>: {data_point["City name"]} {knowledge_base.get(data_point["City name"].lower().strip())}
<assistant>: {data_point["Response"]}
""".strip()
  print(ret)
  return ret

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Dataset({
    features: ['City name', 'Response'],
    num_rows: 51
})


Map:   0%|          | 0/51 [00:00<?, ? examples/s]

<human>: Ahmadabad  name_of_city:Ahmadabad 	state_code:24	state_name:GUJARAT	dist_code:7	population_total:5570585	population_male:2935869	population_female:2634716	0-6_population_total:589076	0-6_population_male:317917	0-6_population_female:271159	literates_total:4464303	literates_male:2459823	literates_female:2004480	sex_ratio:897	child_sex_ratio:853	effective_literacy_rate_total:89.62	effective_literacy_rate_male:93.96	effective_literacy_rate_female:84.81	location:23.022505,72.5713621	total_graduates:769858	male_graduates:435267	female_graduates:334591
<assistant>: Ahmadabad, the Manchester of India, is a vibrant and industrial city, known for its rich cultural heritage and economic significance.
Ahmadabad, with the state code 24 and district code 7, is located in Gujarat. The city has a population of 5,570,585, comprising 2,935,869 males and 2,634,716 females. The youthful demographic includes 589,076 children aged 0-6 years, with 317,917 boys and 271,159 girls. Literacy rates are h

# Finetuning the model

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)
from datasets import load_dataset
data = load_dataset("csv", data_files="spatialdata.csv")
print(data["train"])
def generate_prompt(data_point):

  ret = f"""
<human>: {data_point["City name"]} {knowledge_base.get(data_point["City name"].lower().strip())}
<assistant>: {data_point["Response"]}
""".strip()
  print(ret)
  return ret

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

data = data["train"].shuffle().map(generate_and_tokenize_prompt)
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.946400
2,1.860700
3,1.800800
4,1.740000
5,1.711300
6,1.699400
7,1.662700
8,1.643300
9,1.560600
10,1.632800


TrainOutput(global_step=12, training_loss=1.699753204981486, metrics={'train_runtime': 115.5258, 'train_samples_per_second': 0.441, 'train_steps_per_second': 0.104, 'total_flos': 913760961012480.0, 'train_loss': 1.699753204981486, 'epoch': 0.9411764705882353})

# Save trained model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
notebook_login()

In [ ]:
PEFT_MODEL = "srlog/geodata-falcon-7b-trained"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/srlog/geodata-falcon-7b-trained/commit/1637125c88554a26dede3aed217c82f9274e4e7f', commit_message='Upload model', commit_description='', oid='1637125c88554a26dede3aed217c82f9274e4e7f', pr_url=None, pr_revision=None, pr_num=None)

# Running the finetuned model
The models weights are finetuned and stored as [trained-model](https://drive.google.com/file/d/1xI7o4_jY7GjaA0XuAxCBJAQylHjnSbAb/view?usp=sharing)

Unzip and upload the 'trained-model' folder to the contents folder.



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set the device to CPU
device = "cuda"

# Load tokenizer and base model
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"  # Update with the correct model path
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the base model with float32 precision and force it to CPU
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="cpu"  # Force the model to use CPU
)

# Disable use_cache to avoid issues with gradient checkpointing
base_model.config.use_cache = False

def ask_model(input_text):
    encoding = tokenizer(input_text, return_tensors="pt").to(device)
    # Generation configuration
    generation_config = {
        "max_length": 500,  # Adjust as needed
        "num_return_sequences": 1,
        # Add other necessary parameters
    }

    # Ensure model is in evaluation mode
    base_model.eval()

    with torch.no_grad():
        outputs = base_model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            **generation_config
        )
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Simple loop to interact with the model
while True:
    inp = input("User: ")
    if inp.lower() == 'exit':
        break
    print("Assistant:", ask_model(inp))




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

User: HOw are you feeeling..?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Set the device to CUDA
device = "cuda"

# Load tokenizer and base model
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"  # Update with the correct model path
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the base model with float16 precision for GPU
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Use float16 for GPU efficiency
    device_map="auto"  # Automatically map to CUDA if available
).to(device)  # Ensure the model is on the correct device

# Disable use_cache to avoid issues with gradient checkpointing
base_model.config.use_cache = False

def ask_model(input_text):
    # Move input tensors to the correct device (GPU)
    encoding = tokenizer(input_text, return_tensors="pt").to(device)
    # Generation configuration
    generation_config = {
        "max_length": 500,  # Adjust as needed
        "num_return_sequences": 1,
        # Add other necessary parameters
    }

    # Ensure model is in evaluation mode
    base_model.eval()

    with torch.no_grad():
        # Ensure input_ids and attention_mask are on the correct device
        outputs = base_model.generate(
            input_ids=encoding.input_ids.to(device),
            attention_mask=encoding.attention_mask.to(device),
            **generation_config
        )
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Simple loop to interact with the model
while True:
    inp = input("User: ")
    if inp.lower() == 'exit':
        break
    print("Assistant:", ask_model(inp))


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

...

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Assistant: hey there,
i'm looking for a new job.
i'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm looking for a new job.
I'm lo

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch

import pandas as pd

# Create a DataFrame using the provided dataset
data = pd.DataFrame({
    'text': [
        "I'm feeling really down today, nothing seems to be going right.",
        "The loss of my pet has left me devastated.",
        "Life feels so heavy, I can't stop crying.",
        "I can't believe this is happening, everything is falling apart.",
        "I feel so alone and hopeless right now.",
        "This is the worst day of my life, I can't see a way out.",
        "My heart is broken, I don't know how to move on.",
        "I never thought I could feel this much pain.",
        "Everything feels pointless, I'm overwhelmed with sorrow.",
        "I just want to disappear, this sadness is too much to bear.",
        "I just got a promotion at work, I'm so happy!",
        "Today has been the best day ever, I feel on top of the world!",
        "I'm feeling so grateful for everything in my life.",
        "I'm so excited, I can't stop smiling!",
        "I feel so blessed to have such amazing friends and family.",
        "The sunshine makes everything feel so perfect today.",
        "I'm thrilled about my upcoming vacation, I can't wait!",
        "I just received some great news, I'm so happy right now!",
        "Life is beautiful, I feel so joyful and content.",
        "Everything is going so well, I'm overjoyed!",
        "I'm head over heels in love with you.",
        "You make my world a better place, I'm so lucky to have you.",
        "I can't stop thinking about you, you're always on my mind.",
        "Every moment with you feels magical, I love you so much.",
        "I'm so in love, it's the best feeling in the world.",
        "I never knew love could be this amazing until I met you.",
        "You are my soulmate, I cherish every moment with you.",
        "I'm so grateful for your love and support, it means the world to me.",
        "I love you more than words can express, you are my everything.",
        "Being with you is the best part of my day, I love you endlessly.",
        "I am so furious, I can't believe you did that!",
        "This situation makes me so angry, I'm about to explode!",
        "I can't stand this anymore, I'm absolutely livid!",
        "How dare you treat me like this, I'm beyond mad!",
        "I'm so frustrated, this is completely unacceptable!",
        "You have no right to make me feel this way, I'm so angry!",
        "I'm about to lose my temper, this is ridiculous!",
        "This injustice is infuriating, I can't stay quiet any longer.",
        "I've had enough of this nonsense, I'm so mad right now!",
        "I can't believe how angry I am, this is outrageous!",
        "I'm terrified of what's going to happen next.",
        "This situation is making me so anxious, I'm really scared.",
        "I'm afraid that everything is going to go wrong.",
        "I can't shake this feeling of dread, something bad is going to happen.",
        "I'm so worried, I feel paralyzed by fear.",
        "The thought of losing everything terrifies me.",
        "I'm scared of what the future holds, I can't stop thinking about it.",
        "This fear is overwhelming, I don't know how to deal with it.",
        "I'm so nervous, I feel like something bad is going to happen.",
        "This is the scariest thing I've ever experienced.",
        "Wow, I can't believe that just happened, what a surprise!",
        "I'm shocked, I never saw that coming!",
        "This is unbelievable, I'm completely taken aback!",
        "I didn't expect that at all, what a surprise!",
        "I can't believe my eyes, this is so unexpected!",
        "This news is so surprising, I'm still processing it.",
        "I'm in shock, I never imagined this would happen!",
        "What a pleasant surprise, I didn't see that coming!",
        "I'm stunned, this is the last thing I expected.",
        "This is so unexpected, I can't believe it!"
    ],
    'label': [
        'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness', 'sadness',
        'sadness', 'sadness', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy',
        'joy', 'joy', 'love', 'love', 'love', 'love', 'love', 'love', 'love', 'love',
        'love', 'love', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger',
        'anger', 'anger', 'anger', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear',
        'fear', 'fear', 'fear', 'fear', 'surprise', 'surprise', 'surprise', 'surprise',
        'surprise', 'surprise', 'surprise', 'surprise', 'surprise', 'surprise'
    ]
})

print(data.head())  # Display the first few rows of the DataFrame


# Map labels to numeric values
label_mapping = {label: idx for idx, label in enumerate(data['label'].unique())}
data['label'] = data['label'].map(label_mapping)

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(data['text'], data['label'], test_size=0.2)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Step 2: Prepare Dataset for PyTorch
class EmotionDataset(torch.utils.data.Dataset):
    def _init_(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def _getitem_(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def _len_(self):
        return len(self.labels)

# Create PyTorch datasets
train_dataset = EmotionDataset(train_encodings, list(train_labels))
val_dataset = EmotionDataset(val_encodings, list(val_labels))

# Step 3: Build and Train the BERT Model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_mapping))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

# Step 4: Evaluate the Model
trainer.evaluate()

# Step 5: Save the Model
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

print("Model training and evaluation completed. Model saved to './saved_model'")

                                                text    label
0  I'm feeling really down today, nothing seems t...  sadness
1         The loss of my pet has left me devastated.  sadness
2          Life feels so heavy, I can't stop crying.  sadness
3  I can't believe this is happening, everything ...  sadness
4            I feel so alone and hopeless right now.  sadness


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

TypeError: EmotionDataset() takes no arguments

In [ ]:

def ask_model(input_text):
    # Move input tensors to the correct device (GPU)
    encoding = tokenizer(input_text, return_tensors="pt").to(device)
    # Generation configuration
    generation_config = {
        "max_length": 500,  # Adjust as needed
        "num_return_sequences": 1,
        # Add other necessary parameters
    }

    # Ensure model is in evaluation mode
    base_model.eval()

    with torch.no_grad():
        # Ensure input_ids and attention_mask are on the correct device
        outputs = base_model.generate(
            input_ids=encoding.input_ids.to(device),
            attention_mask=encoding.attention_mask.to(device),
            **generation_config
        )
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return result

# Simple loop to interact with the model
while True:
    inp = input("User: ")
    if inp.lower() == 'exit':
        break
    print("Assistant:", ask_model(inp))


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Assistant: hey i feel sad.
i'm sorry to hear that. Is there anything I can do to help?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Assistant: I didnt' finish my homeworks for my class today..
I'm sorry to hear that. It's important to complete your assignments on time to maintain good grades. Have you spoken to your teacher about the situation?
User: I got poor grades


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Assistant: I got poor grades in high school and college.
It's never too late to improve your grades. You can start by setting specific goals and creating a study plan to achieve them. You can also seek help from a tutor or mentor to guide you through the process. Additionally, you can try to identify the root cause of your poor grades and work on addressing it. Good luck!


# This is just a Prototype, but for futher versions we llama 3.1 8b model and we planned to add more datasets to the knowledgebase containing all geospatial entities for wider knowledge.

# Generating dataset for finetuning LLM

In [ ]:

import pandas as p
!pip install groq
from groq import Groq


conversation_history = [
    {
        "role": "system",
        "content": """You should only contain the expected response message and the answer should strictly be in that format. You are a Context aware Geosptatial data retireval model , so what u should do is i will give you some geospatial information about the city and u should give me the summarised information about the city,
        First give a short description about the city and then You should include all the information provided

        For example,
        User: Give a summary about name_of_city:Chennai 	state_code:33	state_name:TAMIL NADU	dist_code:2	population_total:4681087	population_male:2357633	population_female:2323454	0-6_population_total:418541	0-6_population_male:213084	0-6_population_female:205457	literates_total:3850472	literates_male:2004498	literates_female:1845974	sex_ratio:986	child_sex_ratio:964	effective_literacy_rate_total:90.33	effective_literacy_rate_male:93.47	effective_literacy_rate_female:87.16	location:13.0826802,80.2707184	total_graduates:879695	male_graduates:487428	female_graduates:392267
        Expected Response:
        Chennai is a growing metropolis and the capital city of Tamil Nadu, known for its rich cultural heritage and economic significance.
        Chennai, with the state code 33 and district code 2, is located in Tamil Nadu. The city has a total population of 4,681,087, comprising 2,357,633 males and 2,323,454 females. The city has a young population of 418,541 children aged 0-6 years, with 213,084 boys and 205,457 girls. Literacy is a strong suit for Chennai, boasting a total of 3,850,472 literates, with 2,004,498 males and 1,845,974 females. The city has a sex ratio of 986 females for every 1,000 males and a child sex ratio of 964 girls for every 1,000 boys. The effective literacy rate stands at an impressive 90.33%, with male literacy at 93.47% and female literacy at 87.16%. Geographically, Chennai is situated at 13.0826802 latitude and 80.2707184 longitude. Additionally, the city is home to 879,695 graduates, including 487,428 males and 392,267 females, highlighting its focus on higher education and skilled workforce.

        User: Give a summary abutname_of_city:Greater Mumbai 	state_code:27	state_name:MAHARASHTRA	dist_code:99	population_total:12478447	population_male:6736815	population_female:5741632	0-6_population_total:1139146	0-6_population_male:599007	0-6_population_female:540139	literates_total:10237586	literates_male:5727774	literates_female:4509812	sex_ratio:852	child_sex_ratio:902	effective_literacy_rate_total:90.28	effective_literacy_rate_male:93.32	effective_literacy_rate_female:86.7	location:19.0760,72.8777	total_graduates:1802371	male_graduates:964964	female_graduates:837407
        Expected Response:
        Greater Mumbai, the bustling financial capital of India, is renowned for its vibrant culture and economic vitality.
        Greater Mumbai, with the state code 27 and district code 99, is located in Maharashtra. The city has a substantial population of 12,478,447, comprising 6,736,815 males and 5,741,632 females. The youthful demographic includes 1,139,146 children aged 0-6 years, with 599,007 boys and 540,139 girls. Literacy rates are high, with 10,237,586 literates, including 5,727,774 males and 4,509,812 females. The city's sex ratio is 852 females per 1,000 males, and the child sex ratio is 902 girls per 1,000 boys. The effective literacy rate stands at 90.28%, with males at 93.32% and females at 86.7%. Geographically, Greater Mumbai is situated at 19.0760 latitude and 72.8777 longitude. The city is also home to 1,802,371 graduates, with 964,964 males and 837,407 females, highlighting its significant educational achievements and emphasis on a skilled workforce.
        """
    }
]
def ask(user_input,key):
    client = Groq(
      api_key=key)
    conversation_history.append({
        "role": "user",
        "content": user_input
    })
    chat_completion = client.chat.completions.create(
        messages=conversation_history,
        model="llama-3.1-8b-instant",
        temperature=0.5,
        max_tokens=1024,
        top_p=1,
        stop=None,
        stream=False,
    )

    response = chat_completion.choices[0].message.content
    conversation_history.append({
        "role": "assistant",
        "content": response
    })
    return response

key = "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"

df = p.read_csv("cities_r2.csv")
#So we are taking the first 50 most populated cities to train our LLM model
df.sort_values("population_total",ascending=False,inplace=True)
data = {
    "City name":[],
    "Response":[]
}
import time
for i in range(32,51):
  name = (df.iloc[i][0])
  prompt = f"""
  name_of_city:{df.iloc[i][0]}	state_code:{df.iloc[i][1]}	state_name:{df.iloc[i][2]}	dist_code:{df.iloc[i][3]}	population_total:{df.iloc[i][4]}	population_male:{df.iloc[i][5]}	population_female:{df.iloc[i][6]}	0-6_population_total:{df.iloc[i][7]}	0-6_population_male:{df.iloc[i][8]}	0-6_population_female:{df.iloc[i][9]}	literates_total:{df.iloc[i][10]}	literates_male:{df.iloc[i][11]}	literates_female:{df.iloc[i][12]}	sex_ratio:{df.iloc[i][13]}	child_sex_ratio:{df.iloc[i][14]}	effective_literacy_rate_total:{df.iloc[i][15]}	effective_literacy_rate_male:{df.iloc[i][16]}	effective_literacy_rate_female:{df.iloc[i][17]}	location:{df.iloc[i][18]}	total_graduates:{df.iloc[i][19]}	male_graduates:{df.iloc[i][20]}	female_graduates:{df.iloc[i][21]}
  """
  response = ask(prompt,key)

  data["City name"].append(name)
  data["Response"].append(response)
  print(name)
  print(response)
  break

d = p.DataFrame(data)
d.to_csv("spatialdata.csv")
print("Dataset created")

Dataset created
